In [ ]:
#installs telebot library using pip

#!pip install telebot
#https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
#use the below method to pip install telebot instead, refer to above link for explanation

# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install telebot
!{sys.executable} -m pip install opencv-python

In [ ]:
#import libraries

#image processing library
import cv2
#telegram bot library
import telebot
#threading library, for multiple flows of code execution
import threading
#numpy library, a numerical computing python library
import numpy

In [ ]:
#stores formatted telebot messages in txt file
def archive_message(message) -> None:

In [ ]:
#global variable declaration
image_counter = 0

In [ ]:
#starts telegram bot

#create telegram bot
bot = telebot.TeleBot("<fill in Telegram Bot API key>")

In [ ]:
#default reply to message, echoes the text back
def default_reply(message):
    bot.reply_to(message, message.text)

In [ ]:
#message handlers

#if this is changed, then need to redeclare the bot object. I suspect it's because the new message handlers do not override
#the previous ones (and they are based on sequential order of declaration with the original message handlers declared first)

#start message handler, called when /start command is sent
@bot.message_handler(commands=['start'])
def send_welcome(message) -> None:
    print (message.text)
    bot.send_message(message.chat.id, "Hi, I'm a Telegram Bot. How are you?")
    
#demonstrate use of regex expression, all characters in string are integer characters
@bot.message_handler(regexp="[0-9]")
def all_numbers(message) -> None:
    bot.reply_to(message, "That is a string of numbers")
    
#default message handler, separated from other handlers because of chat functionality added later
@bot.message_handler(func=lambda message: True)
def echo_all(message) -> None:
    default_reply(message)

In [ ]:
#start polling for messages

#if rerunning this kernel, need to stop for a few seconds to prevent simultaneous requests to the telegram API
bot.polling()

In [ ]:
#IBM Watson Assistant

#create Watson Assistant object
authenticator = IAMAuthenticator('<fill in Watson API Key>')
assistant = AssistantV2(\
    version='2019-02-28',\
    authenticator=authenticator\
)

#function to create IBM Watson Assistant session using Watson Assistant object
def create_session() -> str:
    #return session_id
    session_id = assistant.create_session(assistant_id="<fill in Watson Assistant ID>").get_result()["session_id"]
    return session_id


#set service url for Watson Assistant, based on Assistant URL
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com')

In [ ]:
#setting up Watson API for default message handler
#sort of demonstrates polymorphism, with the initial default_reply having same name as this one
def default_reply(message):
    bot.reply_to(message, "Testing")

In [ ]:
#def get_image function to get image frame from camera
def get_image(camera_index:int=0):
    #read image from camera_index (default 0, i.e. first camera)
    video = cv2.VideoCapture(0)
    ret, frame = video.read()
    return frame

In [ ]:
#get image and print image 

#gets image
frame = get_image()

#displays image
cv2.imshow("test", frame)

#closes image window when key is pressed
cv2.waitKey(0)

In [ ]:
#add message handler to take images
@bot.message_handler(commands=["photo"])
def take_photo(message) -> None:
    frame = get_image()
    #access global image_counter variable
    global image_counter
    #probably not the best solution but it keeps an archive of the images and sends it
    #need to find a better way to send the image directly as numpy array or without saving it
    #this makes it a little slow serving photo requests
    cv2.imwrite(f"temp_image{image_counter}.jpg", frame)
    bot.send_photo(chat_id=message.chat.id, photo=open(f"temp_image{image_counter}.jpg", "rb"))
    image_counter += 1